In [1]:
import json
import facebook
import numpy as np
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
        .builder\
        .appName("facebook")\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://localhost:9084")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .config("spark.sql.catalogImplementation", "hive")\
        .enableHiveSupport()\
        .config("spark.yarn.queue","prod")\
        .getOrCreate()
spark

23/09/05 15:41:19 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.0.41 instead (on interface ens38)
23/09/05 15:41:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 15:41:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/05 15:41:26 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [1]:
import requests

access_token='EAAKh3aJk1o4BOzzQx2hDoMwawGGymavLOd1W9XGyqlaBZBPvWxBIEqlOnldZBxPk1AalXh3ll9ZCxVaQK6UM2ADFBj9AqeJp4y9QtcRE4XZB7GIRxsE6eeHYZCpOhth42RXKr0liESOhRbCsCLwSOVAe8jukr500LbyZBRHzbSCgP5q5B3XPu4pVlZBtSfQAkbD'
page_id='365304090334712'
url = f'https://graph.facebook.com/v17.0/{page_id}/posts?fields=id,message,created_time,permalink_url,full_picture,likes.limit(0).summary(true),comments{{id,message,created_time}}&access_token={access_token}'

response = requests.get(url)
data = response.json()
# Extract posts from the response data
posts = data['data']
c=0
postslist=[]   
commentslist=[]
while 'paging' in data and 'next' in data['paging'] and c <= 2:
    next_url = data['paging']['next']
    response = requests.get(next_url)
    data=response.json()
    posts.extend(data['data'])
    c= c+1

for post in posts:        
    id=post['id']
    message=post.get('message', '')
    created_time=post['created_time']
    full_picture=post.get('full_picture', '')
    permalink_url=post['permalink_url']
    likes=post['likes']['summary']['total_count']
    postslist.extend((id,message,created_time,full_picture,permalink_url,likes))

    # # Extract comments
    if 'comments' in post:
        comments = post['comments']['data']
        for comment in comments:
            id=post['id']
            comments_id=comment['id']
            comments_message=comment.get('message', '')
            comments_created_time=comment['created_time']
            commentslist.extend((id,comments_id,comments_created_time,comments_message))
            
print(postslist)
print(commentslist)

['365304090334712_698770025625333', 'سفرة مارسيليا ❤️\n- تصميم عصري بألوان مميزة 👌 \n- بمقاسات تناسب مختلف المساحات 😍 \n- أطول فترة ضمان على الاخشاب تصل 10 سنوات \nمكونات الغرفة :\n- ترابيزة مقاس ١٧٠×٩٢سم وارتفاع ٧٨سم \n- ارجنتيرة مقاس ٧٢×٤٦×١٩٤سم \n-بوفية مقاس ١٨٢×٤٦سم وارتفاع ٩٠سم \n- عدد ٣ مرآة \n- عدد ٦ كرسي \n🔹متاح تجزئة الغرفة \n🔶متاح التقسيط ١٨ شهر بدون فوائد \nتقدر دلوقتي تطلب المنتج اون لاين : mffco.com او من جميع الفروع \nأثاث من الخشب الطبيعي .. يدوم العمر ❤️\nـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ\nزوروا معارضنا قبل إنتهاء العروض \n📍مدينة نصر ..📍المعادي .. 📍المهندسين .. 📍الهرم ..📍حلوان ..📍الأسكندرية ( سموحة )\nwww.mffco.com\n#Mffco_Helwan\n#مفكو_حلوان\n#اثاث_من_الخشب_الطبيعي', '2023-09-01T12:59:26+0000', 'https://scontent.fcai10-1.fna.fbcdn.net/v/t39.30808-6/374769222_698769872292015_6486824128029331579_n.jpg?stp=dst-jpg_p720x720&_nc_cat=101&ccb=1-7&_nc_sid=49d041&_nc_ohc=IZponrDyQ-EAX9PEMqt&_nc_ht=scontent.fcai10-1.fna&edm=AKIiGfEEAAAA&oh=00_AfB9UZ

In [4]:
data

{'data': [{'id': '365304090334712_1990876901110748',
   'message': '#كونسول_مايا \nأناقة الخشب الطبيعي\nــــــــــــــــــــــــــــــــــــــــــــــــــــ\nزوروا معارضنا قبل إنتهاء العرض \n📍مدينة نصر 📍المعادي 📍المهندسين 📍الهرم 📍حلوان 📍الأسكندرية (سموحة)\n#Mffco_Helwan\n#مفكو_حلوان\n#اثاث_من_الخشب_الطبيعي\n#غرفة_نوم_أطفال',
   'created_time': '2022-09-21T12:00:13+0000',
   'permalink_url': 'https://www.facebook.com/365304090334712/posts/1990876901110748/',
   'full_picture': 'https://scontent.fcai10-1.fna.fbcdn.net/v/t39.30808-6/308080394_1990876827777422_5745589566593518020_n.jpg?stp=dst-jpg_p720x720&_nc_cat=110&ccb=1-7&_nc_sid=5614bc&_nc_ohc=gF5nRxfGeYUAX9dRd-R&_nc_ht=scontent.fcai10-1.fna&edm=AKIiGfEEAAAA&oh=00_AfDNXyWSMNX1C6GblY2N9hy4VlOauId4D7EULi_8anjRNQ&oe=64FCD70E',
   'likes': {'data': [],
    'summary': {'total_count': 120, 'can_like': True, 'has_liked': False}},
   'comments': {'data': [{'id': '1990876901110748_586972599933709',
      'message': 'سعرها',
      'created_time

In [2]:
posts

[{'id': '365304090334712_698770025625333',
  'message': 'سفرة مارسيليا ❤️\n- تصميم عصري بألوان مميزة 👌 \n- بمقاسات تناسب مختلف المساحات 😍 \n- أطول فترة ضمان على الاخشاب تصل 10 سنوات \nمكونات الغرفة :\n- ترابيزة مقاس ١٧٠×٩٢سم وارتفاع ٧٨سم \n- ارجنتيرة مقاس ٧٢×٤٦×١٩٤سم \n-بوفية مقاس ١٨٢×٤٦سم وارتفاع ٩٠سم \n- عدد ٣ مرآة \n- عدد ٦ كرسي \n🔹متاح تجزئة الغرفة \n🔶متاح التقسيط ١٨ شهر بدون فوائد \nتقدر دلوقتي تطلب المنتج اون لاين : mffco.com او من جميع الفروع \nأثاث من الخشب الطبيعي .. يدوم العمر ❤️\nـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ\nزوروا معارضنا قبل إنتهاء العروض \n📍مدينة نصر ..📍المعادي .. 📍المهندسين .. 📍الهرم ..📍حلوان ..📍الأسكندرية ( سموحة )\nwww.mffco.com\n#Mffco_Helwan\n#مفكو_حلوان\n#اثاث_من_الخشب_الطبيعي',
  'created_time': '2023-09-01T12:59:26+0000',
  'permalink_url': 'https://www.facebook.com/697740162394986/posts/698770025625333',
  'full_picture': 'https://scontent.fcai10-1.fna.fbcdn.net/v/t39.30808-6/374769222_698769872292015_6486824128029331579_n.jpg?s

In [74]:
while 'paging' in data and 'next' in data['paging'] and c <= 3:
    next_url = data['paging']['next']
    response = requests.get(next_url)
    data=response.json()
    data['data'].extend(data)
   # posts.extend(data['data'])
data


{'data': [{'id': '365304090334712_372692159595905',
   'message': 'عرض اليوم من #مفكو\n\nسفــرة لـــورا \nبسعـر 14330 جنية !\n\nمكونة من ( بوفية + ارجنتيرة + ترابيزة + 6 كراسي مودرن )\n\nتقدروا تشرفونا في معارضنا في ( حلوان - مدينة نصر - المعادى - الهرم - اسكندرية )\n\nHotline : 19440 !\n\n#عزوة',
   'created_time': '2015-05-01T13:08:25+0000',
   'permalink_url': 'https://www.facebook.com/MffcoHelwanEgypt/photos/a.369934916538296/372692159595905/?type=3',
   'full_picture': 'https://scontent.fcai10-1.fna.fbcdn.net/v/t31.18172-8/11187777_372692159595905_2861417948287411993_o.jpg?stp=dst-jpg_p720x720&_nc_cat=103&ccb=1-7&_nc_sid=9267fe&_nc_ohc=I7hH7ZDar68AX-21dmm&_nc_ht=scontent.fcai10-1.fna&edm=AKIiGfEEAAAA&oh=00_AfAV-dmvrBueOCi4UGO1yQzDVJxapfeou-4of6EbBM9HRA&oe=651EC0B9',
   'likes': {'data': [],
    'summary': {'total_count': 119, 'can_like': True, 'has_liked': False}},
   'comments': {'data': [{'id': '372692159595905_372793699585751',
      'message': 'يا ريت موديلات استيل',
      'cr

23/09/05 18:01:17 WARN TransportChannelHandler: Exception in connection from /192.168.0.41:49448
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:259)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop

In [59]:
arr_p = np.array(postslist)
lenarry = int((len(arr_p))/6)
arr_posts = arr_p.reshape(lenarry,6)
columens=['id','message','created_time','full_picture','permalink_url','likes']
df_posts = spark.createDataFrame(data=arr_posts,schema=columens)
#df_posts.repartition(1).write.csv('/home/hadoop/SparkProjects/Facebook/01-posts.csv',mode='overwrite',header=True)
df_posts.printSchema()

root
 |-- id: string (nullable = true)
 |-- message: string (nullable = true)
 |-- created_time: string (nullable = true)
 |-- full_picture: string (nullable = true)
 |-- permalink_url: string (nullable = true)
 |-- likes: string (nullable = true)



In [ ]:
df_posts.show(truncate=False)

In [64]:
arr_c = np.array(commentslist)
lenarry = int((len(arr_c))/4)
arr_Comments = arr_c.reshape(lenarry,4)
columens=['id','comments_id','comments_created_time','comments_message']
df_comments = spark.createDataFrame(data=arr_Comments,schema=columens)
#df_comments.repartition(1).write.csv('/home/hadoop/SparkProjects/Facebook/02-Comments.csv',mode='overwrite',header=True)
df_comments.printSchema()

root
 |-- id: string (nullable = true)
 |-- comments_id: string (nullable = true)
 |-- comments_created_time: string (nullable = true)
 |-- comments_message: string (nullable = true)



In [65]:
df_comments.show(truncate=False)

+-------------------------------+--------------------------------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|id                             |comments_id                     |comments_created_time   |comments_message                                                                                                                         |
+-------------------------------+--------------------------------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|365304090334712_698770025625333|698770025625333_309356931647422 |2023-09-02T00:54:31+0000|بكام                                                                                                                                     |
|365304090334712_698770025625333|698770025625333_239727935224103 |2023-09-02T13:

In [16]:
Access_Token='EAAKh3aJk1o4BOzzQx2hDoMwawGGymavLOd1W9XGyqlaBZBPvWxBIEqlOnldZBxPk1AalXh3ll9ZCxVaQK6UM2ADFBj9AqeJp4y9QtcRE4XZB7GIRxsE6eeHYZCpOhth42RXKr0liESOhRbCsCLwSOVAe8jukr500LbyZBRHzbSCgP5q5B3XPu4pVlZBtSfQAkbD'
mffco_page_id='365304090334712'
graph = facebook.GraphAPI(Access_Token)
fields='posts{message,id,created_time,likes.limit(0).summary(true),comments{id,message,created_time},full_picture,permalink_url, limit=100}'
page = graph.get_object(id=mffco_page_id,fields=fields)
#.limit(5)

In [ ]:
readjson = json.dumps(page)
json_data = json.loads(readjson)

In [11]:
postslist=[]
for i in json_data['posts']['data']:
    for r in i:
      if r == 'message':
          message= i['message']
      elif r == 'story' : message = i['story']
    id=i['id']
    created_time=i['created_time']
    full_picture=i['full_picture']
    permalink_url=i['permalink_url']
    likes=i['likes']['summary']['total_count']
    postslist.extend((id,message,created_time,full_picture,permalink_url,likes))

TypeError: list indices must be integers or slices, not str

In [ ]:
commentslist=[]
for i  in json_data['posts']['data']:
   try :
      comments= i['comments']
      for n in comments['data']:
            id=i['id']
            comments_id=n['id']
            comments_message=n['message']
            comments_created_time=n['created_time']
            commentslist.extend((id,comments_id,comments_created_time,comments_message))
   except: 
      "no"